# Data Pipeline

Data can be inputted in the form of 
1. OpenStreetMap `power` data
2. Siemens PSS/E power modelling data (TODO)
3. `transnet` parsed `.csv` files (TODO)

This is then converted to a `pandapower` model for classical analysis using python. 

To run topological optimization models, the `pandapower` model is converted to a `networkx` model. This is then converted into a QUBO format, based on the optimization model requirements, which can then be quantum-optimized. 

## Downloading Data from OSM

Open street map data can be received from an API call in the XML format, which is a hierarchical data structure that looks like HTML. Raw XML data can also be downloaded from the [OSM website](https://www.openstreetmap.org/export#map=16/26.0720/-80.2004), which is equivalent to the API call we call below. OSM data is a `node`, `way`, or `relation`. Nodes are the basis for everything and store geospatial data as `lat`, `lon` attributes. Ways are a list of nodes that represent either lines or polygons. Relations are lists of any OSM component and can represent just about anything, like a power plant. 

We only care about the [power](https://openinframap.org/#14.43/26.06828/-80.20208) key, especially generators, substations, lines, transformers, and switches. 

In [84]:
import requests
import xml.etree.ElementTree as ET
import queue
from collections import defaultdict
import sys,os
# import module from enclosing directory
cdir = sys.path[0] + '/..'
if cdir not in sys.path:
    sys.path.append(cdir)
from util import timeIt, Ith

@timeIt
def get_OSM_data(bbox, KEYTAG='power', type_info=True, round_to=7, print_info=True)\
    -> tuple[dict[int,dict], dict[str,list], dict[str,dict]] | tuple[dict[int,dict], dict[str,list]]:
    """ 
    Downloads open street map data within the specified bounding box using the OSM API.
    Only filters items that have the KEYTAG tag. 
    
     INPUT: 
      bbox: tuple of 4 floats (left, bottom, right, top). needed to define rectangular bounding box
      round_to: int. specifies the number of decimal places to round the bounding box to
      print_info: bool. if True, print the number of nodes, ways, and relations found in the bounding box
      type_info: bool. if True, make and return a SUBTAGS dictionary that keeps track of the number of 
                       occurences for each tag within each power type. 
                       allows later printing with print_osm_info below.
      KEYTAG: str. the tag to filter by. default is 'power' bc we care about the electrical grid here.
    
     OUTPUT: a 2 or 3-tuple of DATA dictionary, INDICES dictionary, and maybe SUBTAGS dictionary
      DATA is a dictionary of all KEYTAG related items in the bounding box, so that 
      DATA[id] = {'osm_type': 'node', 'lat': lat, 'lon': lon, tags...}         or
                 {'osm_type': 'way', 'nodes': [n1, n2, ...],... tags}     or 
                 {'osm_type': 'relation', 'members': {'nodes': [n1, n2, ...],'ways': [w1, w2, ...]... },... }
        TODO: better way to store relations?
      INDICES is a dictionary of all KEYTAG tags with their indices in the dictionary.
        example for all tems with a tag that has 'k'=KEYTAG, 'v'='generator':
          INDICES['generator'] = [id1, id2, ...] 
      SUBTAGS is a dictionary of all KEYTAG tags with their subtags and the number of occurences of each subtag.
         this is used for later information printing with print_osm_info below.
         example:   SUBTAGS['generator'] = {'gen:type': {'wind':3,'solar':2}, 'name': {}, ...}
    """
    # store all power data in dictionary
    DATA = {}
    # keep track of all indices with each power tag 
    # to easily access all power plants, for example
    INDICES = defaultdict(list)
    # keep track of the type of each power tag. some things might have 
    # multiple types.
    # keep track of the number of occurences for each tag within 
    # each power type
    if type_info:
        SUBTAGS = defaultdict(dict)

    def update_info_dicts(id, tags):
        ''' update INDICES and SUBTAGS '''
        ptype = tags[KEYTAG]
        INDICES[ptype].append(id)
        # keep track of the number of occurences for each tag within a power type
        if type_info:
            for key, val in tags.items():
                if key not in (KEYTAG, 'lat', 'lon', 'nodes', 'members', 'nnodes'):
                    SUBTAGS[ptype][key] = SUBTAGS[ptype].get(key, defaultdict(int))
                    SUBTAGS[ptype][key][val] += 1
    
    bbox = tuple(map(lambda x: round(x, round_to), bbox))
    Q = queue.Queue()
    Q.put(bbox)
    nbboxes = 1

    nnodes = nways = nrels = 0
    while nbboxes:
        bbox_ = Q.get()
        nbboxes -= 1
        # api request for selected bounding box
        url = "https://api.openstreetmap.org/api/0.6/map?bbox="
        url += ",".join(map(str, bbox_))
        #print(url)
        request = requests.get(url)   # make the api request
        response = request.content    # get the (bytes) response
        try:   # parse the response into a dictionary
            root = ET.fromstring(response)
        except ET.ParseError:
            # we requested too many nodes, so now let's
            # split the bounding box into 4 quadrants
            left, bottom, right, top = bbox_
            mid_x = round((left + right) / 2,round_to)
            mid_y = round((bottom + top) / 2,round_to)
            # add 4 quadrants to the queue
            Q.put((left, bottom, mid_x, mid_y))  # bottom left
            Q.put((mid_x, bottom, right, mid_y)) # bottom right
            Q.put((left, mid_y, mid_x, top))     # top left
            Q.put((mid_x, mid_y, right, top))    # top right
            nbboxes += 4
            continue
        
        # add all KEYTAG related items to dictionary for later use
        for element in root:
            # filter only items with KEYTAG specified
            save = False
            for child in element:
                if child.tag == 'tag':
                    if child.get('k') == KEYTAG:
                        save = True
                        break
            if not save: continue

            # parse element and save to dictionaries
            id = int(element.get('id'))
            if element.tag == 'node':
                tags = {"osm_type": "node", "lat": element.get('lat'), "lon": element.get('lon')}
                for child in element:   # assume all tages are child.tag == 'tag'
                    if child.tag == 'tag':
                        key = child.get('k'); val = child.get('v')
                        if key and val:
                            tags[key] = val
                            if key == KEYTAG: save = True
                        else:
                            print(f"Node {id} has a tag with no key-value: ", child.attrib)
                nnodes += 1
            elif element.tag == 'way':
                tags = {'osm_type': "way", 'nodes': []}
                waylen = 0
                for child in element:
                    if child.tag == 'nd':
                        tags['nodes'].append(child.get('ref'))
                        waylen += 1
                    elif child.tag == 'tag':
                        key = child.get('k'); val = child.get('v')
                        if key and val:
                            tags[key] = val
                        else:
                            print(f"Way {id} has a tag with no key-value: ", child.attrib)
                tags['nnodes'] = waylen
                nways += 1
            elif element.tag == 'relation':
                tags = {"osm_type": "relation", 'members': {}}
                for member in element:
                    if member.tag == 'member':
                        key = member.get('type')+'s'
                        if key not in tags['members']:
                            tags['members'][key] = []
                        tags['members'][key].append(member.get('ref'))
                    elif member.tag == 'tag':
                        key = child.get('k'); val = child.get('v')
                        if key and val:
                            tags[key] = val
                        else:
                            print(f"Relation {id} has a tag with no key-value: ", child.attrib)
                
                nrels += 1
            else:
                print("Unknown element: ", element)
                continue
            DATA[id] = tags
            update_info_dicts(id, tags)
    if print_info:
        print(f"Bbox {bbox} has {nnodes} nodes, {nways} ways, and {nrels} relations.")
    if type_info:
        return DATA, INDICES, SUBTAGS
    return DATA, INDICES


def print_osm_info(INDICES, SUBTAGS, KEYTAG='power', indent=4):
    """ prettily prints all openstreetmap information. 
        ensures right-indentation of counts for prettiness via the largest 
        count of a single KEYTAG instance """
    keyinfo = [(key, len(INDICES[key])) for key in SUBTAGS.keys()]
    keyinfo.sort(key=lambda x: x[1], reverse=True)
    alen = len(str(max([count for key, count in keyinfo])))
    for x in keyinfo:
        key, nitems = x
        lev0 = indent + alen - len(str(nitems))
        print(' '*lev0+f"{nitems} {KEYTAG}={key} has {len(SUBTAGS[key])} diff tags:")
        taginfo = [(tag, tagdict, sum(c for c in tagdict.values())) for tag, tagdict in SUBTAGS[key].items()]
        taginfo.sort(key=lambda x: x[2], reverse=True)
        for tag, tag_counts, num_appearances in taginfo:
            num_different = len(tag_counts)
            names = [(name, count) for name, count in tag_counts.items()]
            names.sort(key=lambda x: x[1], reverse=True)
            lev1 = indent + lev0 + alen - len(str(num_appearances))
            if num_different == 1:
                print(' '*lev1+f"{num_appearances} {tag} = {''.join([k for k in tag_counts.keys()])}")
                continue
            if num_different > 10:  # usually only for the 'name' tag
                print(' '*lev1+f"{num_appearances} {tag} has {num_different} diff. values:")
                print(' '*(lev1+indent)+", ".join([f"({count}) {name}" for name, count in names]))
                continue
            print(' '*lev1+f"{num_appearances} {tag} =")
            for name, count in names:
                # right-align the counts
                clen = len(str(count))
                lev2 = indent + lev1 + alen - clen
                print(' '*lev2+f"{count} {name}")

In [85]:
#bounding box coords around Ft. Lauderdale, FL, power plant
# specified as (left, bottom, right, top) like drawing a rectangle between
# pts from the bottom left corener to the top right corner
ft_lauderdale = (-80.219148, 26.045381, -80.169414, 26.087833)
#print('Getting data for Ft. Lauderdale power plant in Florida')
Df, If, Tf = get_OSM_data(ft_lauderdale, print_info=True)
print_osm_info(If, Tf)

Bbox (-80.219148, 26.045381, -80.169414, 26.087833) has 2603 nodes, 138 ways, and 0 relations.
get_OSM_data ran in 9.312308s
    1397 power=pole has 3 diff tags:
        1397 osm_type = node
         205 material = concrete
           2 line_management = branch
    1081 power=tower has 2 diff tags:
        1081 osm_type = node
           4 line_management = branch
     125 power=line has 12 diff tags:
          125 osm_type = way
          125 voltage =
                64 230000
                61 138000
           86 cables =
                 69 3
                 17 6
           85 frequency = 60
           80 operator = Florida Power & Light
           80 operator:wikidata = Q1430055
           61 name has 18 diff. values:
               (16) Lauderdale Power Sub FPL to Laudania Sub FPL 230KV, (5) Timberlake Sub FPL 230KV, (5) Lauderdale Power Sub FPL 138KV, (5) Lauderdale Power Sub FPL to Ives Sub FPL 138KV, (5) Lauderdale Power Sub FPL to Beverly Sub FPL 138KV, (4) Lauderdale Powe

In [86]:
print('Getting data for Shiloh wind farm in California')
shiloh_wind = (-121.9452,38.0780,-121.7295,38.2452)
D, I, T = get_OSM_data(shiloh_wind, print_info=True)
print_osm_info(I,T)

Getting data for Shiloh wind farm in California
Bbox (-121.9452, 38.078, -121.7295, 38.2452) has 4049 nodes, 141 ways, and 8 relations.
get_OSM_data ran in 3.698179s
    1850 power=pole has 1 diff tags:
        1850 osm_type = node
    1649 power=tower has 4 diff tags:
        1649 osm_type = node
          26 design =
                20 flag
                 6 triangle
          26 structure = tubular
           5 size = huge
     529 power=generator has 13 diff tags:
          529 osm_type =
               528 node
                 1 way
          529 generator:method =
               526 wind_turbine
                 2 combustion
                 1 photovoltaic
          529 generator:source =
               526 wind
                 2 gas
                 1 solar
          528 generator:output:electricity =
               173 2 MW
               139 1.8 MW
               111 1.5 MW
                50 2.3 MW
                29 3 MW
                23 0.66 MW
                 2 47 MW

### OSM Grid Inference

To convert this data to pandapower, we want to model all generators, power plants, and substations as effectively a single substation bus, identified by the keys `generator`, `plant`, `station`, `substation`, and `sub_station`. We then want to connect these busses by transmission lines, which are identified by the power key `line`, `minor_line`, or `cable` and are open ways spanned by towers, portals, pylons, and poles. To be exact, we can add in `transformer` and `switch` components at substations that connect to relevant busses. 

This inference is difficult. OpenGridMap's [transnet](https://github.com/OpenGridMap/transnet/tree/master/app) tackles this problem and stored all of the grid models of 2019 in the transnet-models [repo](https://github.com/OpenGridMap/transnet-models). Instead of doing this inference ourselves, we can connect to these models by downloading them in the `data/transnet` folder. 

The `transnet-models` dataset hosts all inferred peer-to-peer (p2p) circuits found on open street maps up to 2019. A p2p circuit treats substations as nodes connected by transmission lines while ignoring intermediate components such as transformers and switches. This strips any extra information from the circuits, including geodata.


In [11]:
# TODO: implement a baby transnet version...?

# let's just download the transnet data for now.
# we can download data from the transnet models and compare the two
import sys, os
import pandas as pd
import pandapower as pp
import pandapower.auxiliary as aux
pd.options.display.max_rows = 20
from glob import glob
from collections import defaultdict

# get data directory (make if not there)
cwd = os.getcwd()
transnet_dir = cwd + "/../data/transnet/"
if not os.path.exists(transnet_dir):
    os.makedirs(transnet_dir)
# download data
# match a name_lines.csv with a name_nodes.csv

# store a transnet data instance, which is created from reading two .csv files
class TransnetOut():
    def __init__(self, name='', linepath='', nodepath='', read=False):
        """ input paths are paths to .csv files downloaded from transnet """
        self.name = name
        self.linepath = linepath
        self.nodepath = nodepath
        self.dataIsProcessed = False
        if read:
            self.read_data()
    
    def read_data(self, linepath='', nodepath=''):
        """ read the data from the csv files into pandas.Dataframes """
        if not linepath: linepath = self.linepath
        if not nodepath: nodepath = self.nodepath
        self.lines = pd.read_csv(linepath)
        self.nodes = pd.read_csv(nodepath)
        self.dataIsProcessed = True

    def __str__(self) -> str:
        if self.dataIsProcessed:
            ret = f"{self.name} has {len(self.lines)} lines and {len(self.nodes)} nodes"
            return ret
        return f"{self.name} at linepath='{self.linepath.split('/')[-1]}' and nodepath='{self.nodepath.split('/')[-1]}'"

    def __repr__(self) -> str:
        return self.__str__()

# get transnet data from folder
transnet_data = defaultdict(TransnetOut)
for path in glob(transnet_dir + "*.csv"):
    name = path.split('/')[-1]
    realname = '_'.join(name.split('_')[:-1])  # like "california"
    if 'lines' in name:
        transnet_data[realname].name = realname
        transnet_data[realname].linepath = path
    elif 'nodes' in name:
        transnet_data[realname].name = realname
        transnet_data[realname].nodepath = path

for t in transnet_data.values():
    print(t)
    t.read_data()
    print(t)

t_cal = transnet_data['california']
n = t_cal.nodes
for k in n.columns:
    if k in ('type','voltage','frequency'):
        print(k, ', '.join(map(str,n[k].unique())))
l = t_cal.lines
for k in l.columns:
    if k in ('type','voltage','cables','frequency'):
        print(k, ', '.join(map(str,l[k].unique())))

print("\n\nnodes")
print(t_cal.nodes.head())
print("\n\nlines")
print(t_cal.lines.head())

def add_transnet_substation_to_pp(net: aux.pandapowerNet, node: pd.Series) -> None:
    """ add a transnet substation to a pandapower network """
    pp.create_bus(net,index=node['n_id'],name=node['name'], 
                  vn_kv=node['voltage'],  # string of voltages like "115000;12000"
                  geodata=(node['latitude'], node['longitude']), 
                  type='b')

def transnet_to_pp(nodes: pd.DataFrame, lines: pd.DataFrame) -> aux.pandapowerNet:
    """ convert transnet data to power plant data """


california at linepath='california_lines.csv' and nodepath='california_nodes.csv'
california has 361 lines and 336 nodes
type substation, generator, plant
voltage 115000;12000, 230000;115000;13200;12000, 115000;12000;7200, 0, 230000;115000, 115000;21000;12000;7200, 115000, 115000;34500, 230000;115000;12000;7200, 230000;115000;60000;12000, 115000;12000;6600, 115000;60000;13200, 138000, 230000;138000, 138000;34500, 138000;69000, 220000;66000;16000, 220000, 220000;66000;16340, 230000, 220000;66000;12000, 220000;66000;16000;12000, 500000;220000, 220000;66000, 220000;69000, 220000;69000;12000, 230000;12000, 23000;115000;60000;13200;12000;7200, 230000;69000, 230000;60000, 230000;138000;34500, 500000;230000, 500000
frequency nan, 60.0
voltage 115000, 230000, 138000, 220000, 500000
cables 6.0, nan, 3.0, 11.0, 9.0, 12.0
type line, cable
frequency 60.0, nan, 0.0


nodes
        n_id   longitude   latitude        type                    voltage  \
0   35656716 -122.148605  37.710544  substation  

## PSS/E Conversion

The Siemens PSS/E Format is widely used by electrical grid operators around the United States, and is meant to be used with their software (which is $7,600). There seems to be no open-source code to translate between this data to a unified grid model that can be translated to other data formats.

See the files in `data/WECC_240bus*` folder. 
The `.raw` file contains all of the grid information including a load snapshot, 
and the `dyr` contains the information for transient stability modelling, [whatever that is](https://www.powerworld.com/WebHelp/Content/MainDocumentation_HTML/Transient_Stability_Data_GEDYD.htm).

For real-time power flow analysis, the `dyr` file is needed, as this specifies exactly how generators, transformers, and other complex electrical machines interact with voltage, current, and power. The `dyr` file contains a list of specifications of power components of space-separated varaibles of the form `id`, `model name`, `component type`, and then all `variables` necessary to fully model the component. These specifications but does not have its own manual in the PSSE Dynamics system (at least online). However, the dynamic systems are unified across the power modelling community, and good documentation can be found by searching for the names of each `dyr`

Dynamic data: [GENROU model](https://www.powerworld.com/WebHelp/Content/TransientModels_HTML/Machine%20Model%20GENROU.htm) for generators with 18 variables, but 14 in provided dyr...
[GAST](https://www.powerworld.com/WebHelp/Default.htm#TransientModels_HTML/Governor%20GAST_PTI%20and%20GASTD.htm?Highlight=GAST) has 12/14?
[SEXS](https://www.powerworld.com/WebHelp/Default.htm#TransientModels_HTML/Exciter%20SEXS_GE.htm?Highlight=SEXS) exitation model - 11 parameters, only6 in dyr?
[HYGOV](https://www.powerworld.com/WebHelp/Default.htm#TransientModels_HTML/Governor%20HYGOV%20and%20HYGOVD.htm?Highlight=HYGOV) governor model, also TGOV


In [77]:
#TODO: store in pandas dataframe instead of dictionary of lists?
import pandas as pd
from collections import defaultdict
import os

# VIA PSS-E v30 user manual

# all measured in power, but refer to different things
# MW :  active power
# MVar : reactive power - megavolt-amperes reactive
# MVA - megavolt-amperes - total apparent power
# pu = define system in terms of a base power/voltage, and then express all other powers as fractions of that base power/voltage
col_defs = {
# bus data
    'baskv': 'bus base voltage (kV)',
    'gl': 'active shunt admittance (MW/V)',
    'bl': 'reactive shunt admittance (Mvar/V)',
    'vm': '|V| (pu)',
    'va': 'V angle (deg)',
# load data
    'pl': 'active power of MVA load (MW)',
    'ql': 'reactive power of MVA load (Mvar)',
    'ip': 'active power of I load (MW,pu)',
    'iq': 'reactive power of I load (Mvar,pu)',
    'yp': 'active power of shunt (MW,pu)',
    'yq': 'reactive power of shunt (Mvar,pu)',
# generator data
    'pg': 'active power of generator (MW)',
    'qg': 'reactive power of generator (Mvar)',
    'qt': 'maximum reactive power of generator (Mvar)',
    'qb': 'minimum reactive power of generator (Mvar)',
    'vs': 'voltage set point of generator (pu)',
    'ireg': 'voltage regulator status of generator',
    'mbase': 'base MVA of generator', # required for switchin studies, fault analysis, and dynamic simulation
    'zr': 'generator impedance real part (pu)',
    'zx': 'generator impedance imaginary part (pu)',
    'rt': 'step-up transformer impedance real part (pu)',
    'xt': 'step-up transformer impedance imaginary part (pu)',
    'gtap': 'transformer off-nominal turns ratio',
    'stat': 'machine status', # 1=in-service, 0=out-of-service
    'pt': 'maximum active power of generator (MW)',
    'pb': 'minimum active power of generator (MW)',
    'o[i]': 'owner number',
    'f[i]': 'fraction of ownership',
# branch data
    'r': 'resistance (pu)',   # Re(Z)
    'x': 'reactance (pu)',    # Im(Z)
    'b': 'susceptance (pu)',  # Im(Y)
    'rate-[i]': 'loading rating (MVA)', # entered as sqrt(3)*E*I*10^-6
    'gi': 'resistance of first bus (pu)',
    'bi': 'reactance of first bus (pu)',
    'gj': 'resistance of second bus (pu)',
    'bj': 'reactance of second bus (pu)',
    'len': 'line length (user-selected units)',
# transformer stuff
    'windv3': 'winding 3 voltage ratio (pu)',
    'nomv3': 'winding 3 nominal voltage (kV)',
    'ang3': 'winding 3-phase shift angle (deg)',
    'rate3-[i]': 'winding 3 power rating (MVA)',
    # 0=no cntrl, +-1 for voltage, +-2 for reactive power, +-3 for active power
    'cod3': 'transformer control mode during PF', 
    'cont3': 'number of bus to be  controlled by transformer',
    'rma3': 'upper limits of tap ratio | 3-voltage (kV) | shift angle',
    'rmi3': 'lower limits of tap ratio | 3-voltage (kV) | shift angle',
    'vma3': 'upper limit of voltage (pu)',
    'vmi3': 'lower limit of voltage (pu)',
    'ntp3': 'number of tap positions available when COD3 is 1|2',
    'tab3': 'index of transformer impedance correction table if impedence is a function of either tap ratio or phase shift angle',
    'cr3': 'load drop compensation resistance (pu)',
    'cx3': 'load drop compensation reactance (pu)',
}

def parse_to_vars(parts: list[str]) -> None:
    """ modifies list of strings in place"""
    for i in range(len(parts)):
        parts[i] = parts[i].strip()
        # strings are encoded as 'STR     '
        if parts[i][0] == "'":
            parts[i] = parts[i][1:-1].strip()
        try:
            parts[i] = int(parts[i])
        except:
            try:
                parts[i] = float(parts[i])
            except:
                
                pass
            

def PSS_raw_reader(filepath, printInfo=False) -> dict[str: pd.DataFrame]:
    with open(filepath, 'r') as f:
        lines = f.readlines()
    DATA = {}
    # might be completely incorrect, idk
    gen_titles = ['I','Name','Gen MW','Gen MVar','Set volt','min MW','Max MW','Min mVar','Max mvar']
    bus_titles=['ID','NAME','BASKV', # bus base voltage
                'IDE', # bus type code. 
    #1=load bus, 2=gen bus, 3=swing bus (V,º), 4=isolated bus
                'AREA', 'ZONE','OWNER',
                'GL',  # MW/V 
                'BL', # Mvar, 1/V. positive for capacitor, negative for reactor or inductor
                'VM','VA','VM1','VA1']
    # just from what I've seen
    sections = ['bus', 'load', 'fixed shunt','generator','branch','system switching device','transformer','area','two-terminal dc','vsc dc','impedance correction','multi-terminal dc','multi-section line','zone','inter-area transfer','owner','facts device','switched shunt','gne','induction machine','substation']
    section = 'system'
    ret = ''
    i = -1
    rowsPerEntry = 0
    allTitles = []  # multi-line entries
    while i + 1 < len(lines):
        i += 1
        line = lines[i]
        
        # SECTION CHANGE
        if line.startswith("0 / "):
            msgs = line[4:].split(',')
            old = msgs[0][7:]  # skip "END OF "
            if len(ret):
                print(old+'\n'+ret)
            ret = ''
            if len(msgs) != 2:
                # we have reached the end -> no new section
                break
            new = msgs[1][7:].lower() # skip " BEGIN "
            section = new[:-6]  # skip " DATA"
            #print('now on section '+section)
            allTitles = [] # reset titles
            rowsPerEntry = 0
            entryIndex = 0
            # set column titles
            if section == 'bus':  # for some reason only the bus section has no titles
                allTitles = [bus_titles]
                rowsPerEntry = 1
            continue
        elif lines[i].startswith('@!'):
            rowsPerEntry += 1
            titles = lines[i][2:].rstrip().split(',')
            # overcome the wierd string formatting they have
            for j in range(len(titles)):
                titles[j]= titles[j].strip()
                if titles[j][0] == "'":
                    titles[j] = titles[j][1:-1].strip()
            allTitles.append(titles)
        elif section == 'system':
            # we don't care about eanything in the intial specifications, 
            # so let's just print em all
            ret += line
        else:
            # PARSE ROW OF DATA
            parts = line.split(',')
            parse_to_vars(parts)
            # ADD TO DATA
            # some data is not specified (like generator data) but DataFrames must take
            # columns that have the same length, so we must fill in the blanks with None if so
            if entryIndex == 0:
                entry = {}
            #print(allTitles, entryIndex, rowsPerEntry)
            titles = allTitles[entryIndex]
            for j in range(len(parts)):
                if j < len(titles):
                    entry[titles[j]] = parts[j]
                else:
                    entry[str(j)] = parts[j]
            entryIndex += 1
            if entryIndex == rowsPerEntry:
                DATA[section] = DATA.get(section, [])
                DATA[section].append(entry)
                entryIndex = 0
    # convert all dictionaries to pandas dataframes
    for section, data in DATA.items():
        DATA[section] = pd.DataFrame(data)
    if printInfo:
        for section, frame in DATA.items():
            print(f"{len(frame)} {section}(s) w/ {len(frame.columns)} columns {', '.join(frame.columns)}")
            #print(frame.loc[0])
            #print(frame.head())
    return DATA

def PSS_dyr_reader(filepath, printInfo=False) -> pd.DataFrame:
    # lines are separated by "/"
    with open(filepath, 'r') as f:
        lines = f.readlines()
    DATA = []
    titles = ['id','model','letter']
    parts = []  # to store a single data entry
    i = -1
    while i + 1 < len(lines):
        i += 1
        line = lines[i].strip()
        # starting messages
        if line.startswith("//"):
            print(line)
            continue
        # SECTION CHANGE
        if line.startswith("/"):
            continue
        # section 1 stores all generator components probably
        # PARSE LINE INTO PARTS
        atContent = finalPart = False
        j = 0
        while j < len(line):
            if line[j] == "/":
                finalPart = True
            elif line[j] in (' ',','):
                atContent = False
            else:
                if not atContent:
                    parts.append('')
                parts[-1] += line[j]
                atContent = True
            j += 1
        # ADD TO DATA IF DONE WITH TING
        if finalPart:
            parse_to_vars(parts)
            # data is id, model name, single-letter, then variables
            entry = {titles[j]: parts[j] for j in range(3)}
            entry['vars'] = parts[3:]
            DATA.append(entry)
            parts = []  # reset parts
    print('found',len(DATA),'entries in dyr file')
    if printInfo:
        for col in titles:
            uniques = DYR[col].unique()
            print(f"{len(uniques)} unique {col}s: {', '.join(map(str,uniques))}")
    return pd.DataFrame(DATA).sort_values('id')


path = '/Users/benkroul/Documents/Physics/womanium/QUANTUM-GRID-OPTIMIZATION/data/WECC_240bus_2018summer_2021_IEEE-NASPI_OSL/240busWECC_2018_PSS.raw'
RAW = PSS_raw_reader(path, True)
print('\nloading dyr file')
DYR = PSS_dyr_reader(path.replace('.raw','.dyr'), True)

SYSTEM-WIDE DATA
 0, 100.00, 34,  ,  ,   / WECC 240 SYSTEM, V0.2, DEVELOPED BY NREL. CONTACT: HAOYU.YUAN@NREL.GOV
This is a reduced WECC 240-bus power system model reflecting WECC generation resource mix in 2018 and representing the summer peak load. It was developed by NREL and contains no CEII.
Reference: H. Yuan, R. S. Biswas, J. Tan and Y. Zhang, "Developing a Reduced 240-Bus WECC Dynamic Model for Frequency Response Study of High Renewable Integration," IEEE/PES Transmission and Distribution Conference and Exposition (T&D), Chicago, IL, USA, 2020.

243 bus(s) w/ 13 columns ID, NAME, BASKV, IDE, AREA, ZONE, OWNER, GL, BL, VM, VA, VM1, VA1
139 load(s) w/ 17 columns I, ID, STAT, AREA, ZONE, PL, QL, IP, IQ, YP, YQ, OWNER, SCALE, INTRPT, DGENP, DGENQ, DGENF
146 generator(s) w/ 28 columns I, ID, PG, QG, QT, QB, VS, IREG, MBASE, ZR, ZX, RT, XT, GTAP, STAT, RMPCT, PT, PB, O1, F1, O2, F2, O3, F3, O4, F4, WMOD, WPF
329 branch(s) w/ 28 columns I, J, CKT, R, X, B, N A M E, RATE1, RATE2, RATE3